In [1]:
import planetary_computer as pc
import pystac_client
from pystac_client import Client
import geopandas as gpd
import rasterio
import rioxarray
from rasterio.windows import Window
from rasterio.mask import mask
import leafmap
from shapely.geometry import box, mapping
from pystac.extensions.eo import EOExtension as eo
import os

In [2]:
def create_grid(bounds, grid_size):
    minx, miny, maxx, maxy = bounds
    grid = []
    x = minx
    while x < maxx:
        y = miny
        while y < maxy:
            grid.append(box(x, y, x + grid_size, y + grid_size))
            y += grid_size
        x += grid_size
    return grid

In [3]:
def get_pv_roi(shp_path, grid_size):
    # 读取shp文件
    gdf = gpd.read_file(shp_path)
    
    # 获取shp文件的边界
    bounds = gdf.total_bounds
    
    # 创建网格
    grid = create_grid(bounds, grid_size)
    
    # 创建一个新的GeoDataFrame来存储bbox
    grid_gdf = gpd.GeoDataFrame({'geometry': grid})
    # 过滤掉不与面矢量相交的bbox
    grid_gdf = grid_gdf[grid_gdf.intersects(gdf.unary_union)]
    
    # 保存结果到新的shp文件
    grid_gdf.to_file('pv_roi_0.1_grid.shp')

In [4]:
def download_planetary_computer_image(bbox, time_range, cloud_cover_threshold=10, band="visual", output_file="output.tif"):
    """
    下载指定区域和时间范围内的Planetary Computer影像数据。

    参数：
    - bbox: 研究区域的边界框，格式为 [min_lon, min_lat, max_lon, max_lat]
    - time_range: 时间范围，格式为 "YYYY-MM-DD/YYYY-MM-DD"
    - cloud_cover_threshold: 云覆盖率阈值，默认为10%
    - band: 要下载的波段，默认为 "visual"
    - output_file: 输出文件名，默认为 "output.tif"
    """
    # 打开STAC客户端
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=pc.sign_inplace,
    )

    # 搜索数据
    search = catalog.search(
        collections=["sentinel-2-l2a"],
        bbox=bbox,
        datetime=time_range,
        query={"eo:cloud_cover": {"lt": cloud_cover_threshold}},
    )
    items = search.item_collection()

    # 选择云覆盖率最低的数据项
    from pystac.extensions.eo import EOExtension as eo
    selected_item = min(items, key=lambda item: eo.ext(item).cloud_cover)

    # 读取影像波段并导出
    xds = rioxarray.open_rasterio(selected_item.assets[band].href)

    # 创建裁剪用的GeoDataFrame
    geom = box(*bbox)
    gdf = gpd.GeoDataFrame({"geometry": [geom]}, crs="EPSG:4326")

    # 裁剪影像
    xds_clipped = xds.rio.clip(gdf.geometry.apply(mapping), gdf.crs)

    xds_clipped.rio.to_raster(output_file, compress='LZW', tiled=True, dtype="uint8")

    print(f"影像已下载并保存为 {output_file}")



In [5]:
def download_index(roi,out_dir,time_range="2023-01-01/2023-12-31",index_range=None):
    roi_gdf = gpd.read_file(roi)
    if index_range==None:
        index_range = list(range(0, len(roi_gdf)-1))
    for index, row in roi_gdf.iterrows():
        if index in index_range:
            geometry = row['geometry']
            bbox = geometry.bounds
            savefile = os.path.join(out_dir,f"{index}.tif")
            download_planetary_computer_image(bbox=bbox,time_range=time_range, output_file=savefile)

# roi_gdf = gpd.read_file('pv_roi_0.1_grid.shp')
# # 逐个读取要素
# for index, row in roi_gdf.iterrows():
#     geometry = row['geometry']
#     bbox = geometry.bounds
#     savefile="./test/"+f"{index}.tif"

#     if index==8000:
#         print(bbox)
#     # print(f"Feature {index}: {geometry}")
#         download_planetary_computer_image(bbox=bbox,time_range="2023-01-01/2023-12-31", output_file=savefile)



In [6]:
if __name__ == "__main__":
    shp_path = '../Data/Road/GRIP4_density_total/pv_roi/pv_roi.shp'
    grid_size = 1  # 根据需要调整网格大小
    # 需要约80分钟
    # get_pv_roi(shp_path, grid_size)
    roi = 'pv_roi_1_grid.shp'
    # index_range = list(range(8000, 10000))
    download_index(roi, out_dir='E:\PV_Global\data\sentinel2_2023_pc')

影像已下载并保存为 E:\PV_Global\data\sentinel2_2023_pc\0.tif
影像已下载并保存为 E:\PV_Global\data\sentinel2_2023_pc\1.tif
影像已下载并保存为 E:\PV_Global\data\sentinel2_2023_pc\2.tif
影像已下载并保存为 E:\PV_Global\data\sentinel2_2023_pc\3.tif
影像已下载并保存为 E:\PV_Global\data\sentinel2_2023_pc\4.tif
影像已下载并保存为 E:\PV_Global\data\sentinel2_2023_pc\5.tif
影像已下载并保存为 E:\PV_Global\data\sentinel2_2023_pc\6.tif
影像已下载并保存为 E:\PV_Global\data\sentinel2_2023_pc\7.tif
影像已下载并保存为 E:\PV_Global\data\sentinel2_2023_pc\8.tif
影像已下载并保存为 E:\PV_Global\data\sentinel2_2023_pc\9.tif
影像已下载并保存为 E:\PV_Global\data\sentinel2_2023_pc\10.tif
影像已下载并保存为 E:\PV_Global\data\sentinel2_2023_pc\11.tif
影像已下载并保存为 E:\PV_Global\data\sentinel2_2023_pc\12.tif
影像已下载并保存为 E:\PV_Global\data\sentinel2_2023_pc\13.tif
影像已下载并保存为 E:\PV_Global\data\sentinel2_2023_pc\14.tif
影像已下载并保存为 E:\PV_Global\data\sentinel2_2023_pc\15.tif
影像已下载并保存为 E:\PV_Global\data\sentinel2_2023_pc\16.tif
影像已下载并保存为 E:\PV_Global\data\sentinel2_2023_pc\17.tif
影像已下载并保存为 E:\PV_Global\data\sentinel2_2023_pc\18.tif
影像已

APIError: The request exceeded the maximum allowed time, please try again. If the issue persists, please contact planetarycomputer@microsoft.com.

